# 科 share - 匯入網站之課表資料格式處理

In [1]:
import numpy as np
import pandas as pd
import re

## 學生課表

In [2]:
data = pd.read_excel('./data/範例課表.xlsx')
data.head()

,姓名：杜葳葳 科管院學士班21級 第一專長 ：計量財務金融 第二專長：資訊工程,Unnamed: 1,Unnamed: 2,Unnamed: 3
0,學年,學期,科號,科目名稱
1,109,20,CS 550000,計算機圖學
2,109,20,FL 406500,口譯
3,109,20,GE 102400,產業創新實作 -- 社會科學領域 Elective GE course: Social Sc...
4,109,20,IEEM612400,大數據分析與資料挖礦


* 取出姓名當成 output 的檔名
* 入學學年 enrollment_year : 取學年度最小值

In [3]:
name = data.columns[0]
data = data.set_axis(['學年', '學期', '科號', '科目名稱'], axis=1, inplace=False).drop([0])
enrollment_year = min(data['學年'])

In [4]:
def delete_last_two_digits(num):
    if re.search(r'\d+', num) is not None:
        integer_parts = re.search(r'\d+', num).group()
        if len(integer_parts) > 4:
            return num[:-2]
    return num
def delete_space(num):
    return num.replace(" ", "")
def shorten_course_name(text):
    return text.split('--', 1)[0]
  
## 科號移除後兩碼和空格, 科目名稱去除--之後的文字
data['科號'] = data['科號'].apply(delete_space).apply(delete_last_two_digits)
data['科目名稱'] = data['科目名稱'].apply(shorten_course_name)

## 刪除服學(Z)和體育(PE)
for i, value in enumerate(data['科號']):
    if str(value.strip()[0]) == 'Z' or str(value.strip()[0:2]) == 'PE':
        data = data.drop(index = [i+1])

In [5]:
data

,學年,學期,科號,科目名稱
1,109,20,CS5500,計算機圖學
2,109,20,FL4065,口譯
3,109,20,GE1024,產業創新實作
4,109,20,IEEM6124,大數據分析與資料挖礦
5,109,20,IPHD6010,印度社會專題研究
...,...,...,...,...
72,106,10,ECON1001,經濟學原理一
73,106,10,LANG1010,英文一
74,106,10,LST3503,法學緒論
75,106,10,MATH1010,微積分Ａ一


## 課程規則
* 匯入 **科號規則**
* 把科號轉成 list

In [6]:
rule = pd.read_excel('./data/專長課程代號.xlsx')
rule['科號'] = rule['科號'].apply(delete_last_two_digits).apply(delete_space)

def multiple_num(nums):
    return nums.split(',')
def delete_digits(integer_list):
    return list(map(delete_last_two_digits, integer_list))

rule['科號'] = rule['科號'].apply(multiple_num).apply(delete_digits)

In [7]:
rule

,學年度,類別,專長名稱,科目名稱,科號,Unnamed: 5
0,106,基礎必修,NaN,經濟學原理一,[ECON1001],NaN
1,106,基礎必修,NaN,經濟學原理二,[ECON1002],NaN
2,106,基礎必修,NaN,會計學一,"[ECON1271, QF1004]",NaN
3,106,基礎必修,NaN,會計學二,"[ECON1272, QF1005]",NaN
4,106,基礎必修,NaN,財務管理,[QF2141],NaN
...,...,...,...,...,...,...
472,107,第二專長,電機工程,電磁波,[EE3150],NaN
473,107,第二專長,電機工程,計算機網路概論,[EE3650],NaN
474,107,第二專長,電機工程,電子電路實驗,"[EE2245, EE2245]",NaN
475,107,第二專長,電機工程,邏輯設計實驗,"[EE2230, EE2230]",NaN


## 產生網站用課表

In [8]:
output = pd.DataFrame(np.full((data.shape[0], 5), np.nan), columns = ['編號', '課名', '學年度', '學期', '類別'])

def find_specialty(name):
    first_specialty, _ , second_specialty = name[name.find('第一專長 ：')+6:].partition('\u3000')
    second_specialty, _ , _ = second_specialty[second_specialty.find('第二專長：')+5:].partition('\u3000')
    return first_specialty, second_specialty

first_specialty, second_specialty = find_specialty(name)

count = 0

print('學生姓名: ', name[3:6])
print('學生入學年度:', enrollment_year)
print('第一專長:', first_specialty)
print('第二專長:', second_specialty)
    
for i in range(data.shape[0]):
    c_year = data['學年'].iloc[i]
    c_semester = data['學期'].iloc[i]  
    c_num = data['科號'].iloc[i].strip()
    c_name = data['科目名稱'].iloc[i]
    
    for j in range(rule.shape[0]):
        if count > i: ## find the match source
            break
            
        r_num = rule['科號'].iloc[j]
        r_cat = rule['類別'].iloc[j]
        r_specialty = rule['專長名稱'].iloc[j]
            
        for k in range(len(r_num)):
            if str(c_num) == str(r_num[k].replace(' ', '')):
                output['課名'].iloc[i] = c_name
                output['學年度'].iloc[i] = int(c_year)
                output['學期'].iloc[i] = int(c_semester)
               
                if str(r_cat) == '基礎必修':
                    count += 1
                    output['類別'].iloc[i] = r_cat
                    break

                elif str(r_cat) == '第一專長':
                    if r_specialty == first_specialty: ## 一專課程
                        count += 1
                        output['類別'].iloc[i] = r_cat
                        break
                        
                elif str(r_cat) == '第二專長':
                    if str(r_specialty) == str(second_specialty): ## 二專課程
                        count += 1
                        output['類別'].iloc[i] = r_cat
                        break
            
        if j == rule.shape[0]-1: ## 其他課程
            count += 1
            output['課名'].iloc[i] = c_name
            output['學年度'].iloc[i] = int(c_year) 
            output['類別'].iloc[i] = '其他'
            output['學期'].iloc[i] = int(c_semester)

學生姓名:  杜葳葳
學生入學年度: 106
第一專長: 計量財務金融
第二專長: 資訊工程


/Users/user/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


* 匯出檔案格式處理

In [9]:
def format(year):
    return int(year)

output['學年度'] = output['學年度'].apply(format)
output['學期'] = output['學期'].apply(format)

* 由「學年度」與「學期」計算「編號」

In [10]:
match = {
    '基礎必修' : 1 ,
    '第一專長' : 2 ,
    '第二專長' : 3 ,
    '其他' : 4 
}

output['編號'] = output['類別'].map(match)

print('學生入學年度:', enrollment_year)
for i in range(output.shape[0]):
    if output['學年度'].iloc[i] == enrollment_year:
        if output['學期'].iloc[i] == 10:
            output['編號'].iloc[i] = output['編號'].iloc[i]
        else: ## '20' or others
            output['編號'].iloc[i] += 4
    elif output['學年度'].iloc[i] == enrollment_year+1:
        if output['學期'].iloc[i] == 10:
            output['編號'].iloc[i] += 8
        else: ## '20' or others
            output['編號'].iloc[i] += 12
    elif output['學年度'].iloc[i] == enrollment_year+2:
        if output['學期'].iloc[i] == 10:
            output['編號'].iloc[i] += 16
        else: ## '20' or others
            output['編號'].iloc[i] += 20
    elif output['學年度'].iloc[i] == enrollment_year+3:
        if output['學期'].iloc[i] == 10:
            output['編號'].iloc[i] += 24
        else: ## '20' or others
            output['編號'].iloc[i] += 28
    else:
        if output['學期'].iloc[i] == 10:
            output['編號'].iloc[i] += 32
        else: ## '20' or others
            output['編號'].iloc[i] += 36

學生入學年度: 106


* 確認編碼無誤

In [11]:
output

,編號,課名,學年度,學期,類別
0,32,計算機圖學,109,20,其他
1,32,口譯,109,20,其他
2,32,產業創新實作,109,20,其他
3,32,大數據分析與資料挖礦,109,20,其他
4,32,印度社會專題研究,109,20,其他
...,...,...,...,...,...
59,1,經濟學原理一,106,10,基礎必修
60,4,英文一,106,10,其他
61,1,法學緒論,106,10,基礎必修
62,2,微積分Ａ一,106,10,第一專長


* 輸出檔案(檔名為學生姓名)

In [12]:
output = output.drop(columns=['學年度', '學期', '類別'])
output.to_excel(name[3:6]+'.xlsx', index = False)